# Filtering data in SQLAlchemy
***

The purpose of these laboratory classes is to familiarize with methods to creating and execute select query with conditions.

The scope of this classes:
- using select() - to creating select query
- using query() - to creating query
- and_() , or_(), in_() - to add conditions to query
- order_by() - to sort results
- label() - to make alias
- limit() - to limit results of query

### Exercise

Use all of these methods to create queries for the test database. Check their execution time using the [profiling and timing code methods](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

For queries:
1. How many categories of films we have in the rental?
2. Display list of categories in alphabetic order.
3. Find the oldest and youngest film in rental.
4. How many rentals were in between 2005-07-01 and 2005-08-01?
5. How many rentals were in between 2010-01-01 and 2011-02-01?
6. Find the biggest payment in the rental.
7. Find all customers from Polend or Nigeria or Bangladesh.
8. Where live staff memebers?
9. How many staff members live in Argentina or Spain?
10. Which categories of the films were rented by clients?
11. Find all categories of films rented in America.
12. Find all title of films where was playe: Olympia Pfeiffer or Julia Zellweger or Ellen Presley

## Prepare the environment

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

From the previous classes we know two methods of creating a database model in SQLAlchemy based on:
- [mapper](https://docs.sqlalchemy.org/en/13/orm/mapping_api.html#sqlalchemy.orm.mapper)
- [Class representation](https://docs.sqlalchemy.org/en/13/orm/tutorial.html)

### Connect to the database

In [2]:
config_PostgreSQL = {
    "database_type": "",
    "user": "",
    "password": "",
    "database_url": "",
    "port": ,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_PostgreSQL)

engine = create_engine(db_string)

# test the connection
try:
    conn = engine.connect()
    print("Connected successfully!")
except Exception as e:
    print("Failed to connect")
    print(f"Error: {e}")

Base = declarative_base()

Connected successfully!


### Initialize mapper operation

We can use a script to initialize mapper operation. Where `dict_table` is the dictionary with tables representation where the key is the name of the table.

In [3]:
from sqlalchemy import MetaData, Table
from sqlalchemy.inspection import inspect

metadata = MetaData()
dict_table = {}

inspector = inspect(engine)

for table_name in inspector.get_table_names():
    dict_table[table_name] = Table(table_name, metadata, autoload_with=engine)

Below line in the script above shows references to the table representation named *category*.

In [4]:
print(repr(dict_table['category']))
display(repr(dict_table['payment']))

Table('category', MetaData(), Column('category_id', INTEGER(), table=<category>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000017ADB060640>, for_update=False)), Column('name', VARCHAR(length=25), table=<category>, nullable=False), Column('last_update', TIMESTAMP(), table=<category>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000017ADB060B20>, for_update=False)), schema=None)


"Table('payment', MetaData(), Column('payment_id', INTEGER(), table=<payment>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000017ADB08D930>, for_update=False)), Column('customer_id', SMALLINT(), ForeignKey('customer.customer_id'), table=<payment>, nullable=False), Column('staff_id', SMALLINT(), ForeignKey('staff.staff_id'), table=<payment>, nullable=False), Column('rental_id', INTEGER(), ForeignKey('rental.rental_id'), table=<payment>, nullable=False), Column('amount', NUMERIC(precision=5, scale=2), table=<payment>, nullable=False), Column('payment_date', TIMESTAMP(), table=<payment>, nullable=False), schema=None)"

If we want youse Object representation we need run script:

In [5]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select, or_, and_, text, Column, Integer, String, Date

session = (sessionmaker(bind=engine))()

Base = declarative_base()

class Category(Base):
    __tablename__ = 'category'
    category_id  = Column(Integer, primary_key=True)
    name = Column(String(50))
    last_update  = Column(Date)
    def __str__(self):
        return 'Category id:{0}\nCategory name: {1}\nCategory last_update: {2}'.format(self.category_id,self.name,self.last_update)

At the moment we are ready to start creating database queries. The advantage of using ORM is that you don't have to rewrite queries when changing the database engine. The disadvantage, however, is that we are limited by query structures imposed by ORM.

If this does not suit us, we can of course run a query written by us:

In [6]:
stmt = 'select * from category'

with engine.connect() as conn:
    results = conn.execute(text(stmt)).fetchall()

print(results)

[(1, 'Action', datetime.datetime(2006, 2, 15, 9, 46, 27)), (2, 'Animation', datetime.datetime(2006, 2, 15, 9, 46, 27)), (3, 'Children', datetime.datetime(2006, 2, 15, 9, 46, 27)), (4, 'Classics', datetime.datetime(2006, 2, 15, 9, 46, 27)), (5, 'Comedy', datetime.datetime(2006, 2, 15, 9, 46, 27)), (6, 'Documentary', datetime.datetime(2006, 2, 15, 9, 46, 27)), (7, 'Drama', datetime.datetime(2006, 2, 15, 9, 46, 27)), (8, 'Family', datetime.datetime(2006, 2, 15, 9, 46, 27)), (9, 'Foreign', datetime.datetime(2006, 2, 15, 9, 46, 27)), (10, 'Games', datetime.datetime(2006, 2, 15, 9, 46, 27)), (11, 'Horror', datetime.datetime(2006, 2, 15, 9, 46, 27)), (12, 'Music', datetime.datetime(2006, 2, 15, 9, 46, 27)), (13, 'New', datetime.datetime(2006, 2, 15, 9, 46, 27)), (14, 'Sci-Fi', datetime.datetime(2006, 2, 15, 9, 46, 27)), (15, 'Sports', datetime.datetime(2006, 2, 15, 9, 46, 27)), (16, 'Travel', datetime.datetime(2006, 2, 15, 9, 46, 27))]


## 1. How many categories of films we have in the rental?

To make basic query `select` we can use script:

In [7]:
# select * from category

mapper_stmt = select(dict_table['category'].columns.name)
print('Mapper select: ')
print(mapper_stmt)

session_stmt = session.query(Category)
print('\nSession select: ')
print(session_stmt)

Mapper select: 
SELECT category.name 
FROM category

Session select: 
SELECT category.category_id AS category_category_id, category.name AS category_name, category.last_update AS category_last_update 
FROM category


```sql
Mapper select:
SELECT category.category_id, category.name, category.last_update 
FROM category

Session select:
SELECT category.category_id AS category_category_id, category.name AS category_name, category.last_update AS category_last_update 
FROM category
```

As can be seen in the case of a query based on the class session, aliases are added to the names of the columns returned. This is the only difference at this stage of building queries.

To run a query based on the select class. As a result of the script, we get a list of tuples representing the values of table rows. Examples:

In [8]:
with engine.connect() as conn:
    result1 = conn.execute(mapper_stmt).fetchall()

print(result1)
count = len(result1)
print(count)

[('Action',), ('Animation',), ('Children',), ('Classics',), ('Comedy',), ('Documentary',), ('Drama',), ('Family',), ('Foreign',), ('Games',), ('Horror',), ('Music',), ('New',), ('Sci-Fi',), ('Sports',), ('Travel',)]
16


This form of results presentation is inconvenient if we use objectivity in all our software. To return results as a class, use the formula

In [9]:
session_results = session_stmt.all()

# all results print
print('All results: ')
print(session_results)

# print information from first category in result list
print('\nFirst category: ')
print(session_results[0])

All results: 
[<__main__.Category object at 0x0000017ADB0A74F0>, <__main__.Category object at 0x0000017ADB0A75B0>, <__main__.Category object at 0x0000017ADB0A7490>, <__main__.Category object at 0x0000017ADB0A6650>, <__main__.Category object at 0x0000017ADB0A6830>, <__main__.Category object at 0x0000017ADB0A70A0>, <__main__.Category object at 0x0000017ADB0A6080>, <__main__.Category object at 0x0000017ADB0A6350>, <__main__.Category object at 0x0000017ADB0A7070>, <__main__.Category object at 0x0000017ADB0A6DD0>, <__main__.Category object at 0x0000017ADB0A48B0>, <__main__.Category object at 0x0000017ADB0A7730>, <__main__.Category object at 0x0000017ADB0A6410>, <__main__.Category object at 0x0000017ADB0A7100>, <__main__.Category object at 0x0000017ADB0A7010>, <__main__.Category object at 0x0000017ADB0A5BD0>]

First category: 
Category id:1
Category name: Action
Category last_update: 2006-02-15 09:46:27


As you can easily see in this case, the overloaded operator operator ** __ str __ **. This approach is very useful in implementing business logic.

If we want to create a query for selected columns then we use the following pattern. In this case, the query will return a list of results in both cases:

In [10]:
mapper_stmt = select(
    dict_table['category'].columns.category_id,
    dict_table['category'].columns.name
)
print('Mapper select: ')
print(mapper_stmt)

session_stmt = session.query(Category.category_id, Category.name)
print('\nSession select: ')
print(session_stmt)

Mapper select: 
SELECT category.category_id, category.name 
FROM category

Session select: 
SELECT category.category_id AS category_category_id, category.name AS category_name 
FROM category


If you want to use object mapping, create a class and set session query in this way:

In [11]:
class Category_filter(Base):
    __tablename__ = 'category'
    __table_args__ = {'extend_existing': True}
    category_id  = Column(Integer, primary_key=True)
    name = Column(String(50))
    def __str__(self):
        return 'Category id:{0}\nCategory name: {1}'.format(self.category_id,self.name)
    
q = session.query(Category_filter)
print(q)

SELECT category.category_id AS category_category_id, category.name AS category_name, category.last_update AS category_last_update 
FROM category


## 2. Display list of categories in alphabetic order.

In both cases it is possible to sort using the `order_by` function. For ascending sorting, the harvest will look like this:

In [12]:
mapper_stmt2 = select(dict_table['category'].columns.category_id, dict_table['category'].columns.name).\
    order_by(dict_table['category'].columns.name)

print('Mapper select: ')
print(mapper_stmt2)

with engine.connect() as conn:
    result2 = conn.execute(mapper_stmt2).fetchall()

print(result2)

Mapper select: 
SELECT category.category_id, category.name 
FROM category ORDER BY category.name
[(1, 'Action'), (2, 'Animation'), (3, 'Children'), (4, 'Classics'), (5, 'Comedy'), (6, 'Documentary'), (7, 'Drama'), (8, 'Family'), (9, 'Foreign'), (10, 'Games'), (11, 'Horror'), (12, 'Music'), (13, 'New'), (14, 'Sci-Fi'), (15, 'Sports'), (16, 'Travel')]


And in reverse:

In [13]:
mapper_stmt2 = select(dict_table['category'].columns.category_id, dict_table['category'].columns.name).\
    order_by(dict_table['category'].columns.name.desc())

print('Mapper select: ')
print(mapper_stmt2)

with engine.connect() as conn:
    result2 = conn.execute(mapper_stmt2).fetchall()

print(result2)

Mapper select: 
SELECT category.category_id, category.name 
FROM category ORDER BY category.name DESC
[(16, 'Travel'), (15, 'Sports'), (14, 'Sci-Fi'), (13, 'New'), (12, 'Music'), (11, 'Horror'), (10, 'Games'), (9, 'Foreign'), (8, 'Family'), (7, 'Drama'), (6, 'Documentary'), (5, 'Comedy'), (4, 'Classics'), (3, 'Children'), (2, 'Animation'), (1, 'Action')]


The same applies to sessions:

In [14]:
# Session select ascending
session_stmt2_asc = session.query(Category_filter.category_id, Category_filter.name).\
    order_by(Category_filter.name.asc())

print('\nSession select asc: ')
print(session_stmt2_asc)
result_asc = session_stmt2_asc.all()
print(result_asc)

# Session select descending
session_stmt2_desc = session.query(Category_filter.category_id, Category_filter.name).\
    order_by(Category_filter.name.desc())

print('\nSession select desc: ')
print(session_stmt2_desc)
result_desc = session_stmt2_desc.all()
print(result_desc)


Session select asc: 
SELECT category.category_id AS category_category_id, category.name AS category_name 
FROM category ORDER BY category.name ASC
[(1, 'Action'), (2, 'Animation'), (3, 'Children'), (4, 'Classics'), (5, 'Comedy'), (6, 'Documentary'), (7, 'Drama'), (8, 'Family'), (9, 'Foreign'), (10, 'Games'), (11, 'Horror'), (12, 'Music'), (13, 'New'), (14, 'Sci-Fi'), (15, 'Sports'), (16, 'Travel')]

Session select desc: 
SELECT category.category_id AS category_category_id, category.name AS category_name 
FROM category ORDER BY category.name DESC
[(16, 'Travel'), (15, 'Sports'), (14, 'Sci-Fi'), (13, 'New'), (12, 'Music'), (11, 'Horror'), (10, 'Games'), (9, 'Foreign'), (8, 'Family'), (7, 'Drama'), (6, 'Documentary'), (5, 'Comedy'), (4, 'Classics'), (3, 'Children'), (2, 'Animation'), (1, 'Action')]


## 3. Find the oldest and youngest film in rental.

We can also use logical conditions, such as: `or_`, `and_`, `in_`

To limit the number of records returned by the database, we can use the `limit` function.

In [16]:
# mapper_stmt3 = text("""
#     SELECT title, year, date FROM (
#         SELECT film.title AS title, film.release_year AS year, rental.return_date AS date 
#         FROM film
#         INNER JOIN inventory ON film.film_id = inventory.film_id
#         INNER JOIN rental ON inventory.inventory_id = rental.inventory_id
#         GROUP BY title, year, date
#     ) AS temp_table
#     WHERE year = (SELECT MAX(film.release_year) FROM film) OR
#           year = (SELECT MIN(film.release_year) FROM film) AND
#           date IS NULL
# """)

# print('Mapper select:')
# print(mapper_stmt3)
# with engine.connect() as conn:
#     result3 = conn.execute(mapper_stmt3).fetchall()
# print(result3)

# Define subqueries for MIN and MAX release years
min_year_subquery = select(dict_table['film'].columns.release_year).order_by(dict_table['film'].columns.release_year.asc()).limit(1).scalar_subquery()
max_year_subquery = select(dict_table['film'].columns.release_year).order_by(dict_table['film'].columns.release_year.desc()).limit(1).scalar_subquery()

# Main query directly filtering by release year with labeled columns
mapper_stmt3 = select(
    dict_table['film'].columns.title.label('title'),
    dict_table['film'].columns.release_year.label('year'),
    dict_table['rental'].columns.return_date.label('date')
    ).select_from(
        dict_table['film']
        .join(dict_table['inventory'], dict_table['film'].columns.film_id == dict_table['inventory'].columns.film_id)
        .join(dict_table['rental'], dict_table['rental'].columns.inventory_id == dict_table['inventory'].columns.inventory_id)
    ).where(
        or_(
            and_(dict_table['film'].columns.release_year == max_year_subquery, dict_table['rental'].columns.return_date == None),
            and_(dict_table['film'].columns.release_year == min_year_subquery, dict_table['rental'].columns.return_date == None)
            )
).group_by(
    dict_table['film'].columns.title,
    dict_table['film'].columns.release_year,
    dict_table['rental'].columns.return_date
)

print('Mapper select:')
print(mapper_stmt3)

with engine.connect() as conn:
    result3 = conn.execute(mapper_stmt3).fetchall()

print(result3)

Mapper select:
SELECT film.title AS title, film.release_year AS year, rental.return_date AS date 
FROM film JOIN inventory ON film.film_id = inventory.film_id JOIN rental ON rental.inventory_id = inventory.inventory_id 
WHERE film.release_year = (SELECT film.release_year 
FROM film ORDER BY film.release_year DESC
 LIMIT :param_1) AND rental.return_date IS NULL OR film.release_year = (SELECT film.release_year 
FROM film ORDER BY film.release_year ASC
 LIMIT :param_2) AND rental.return_date IS NULL GROUP BY film.title, film.release_year, rental.return_date
[('Academy Dinosaur', 2006, None), ('Ace Goldfinger', 2006, None), ('Affair Prejudice', 2006, None), ('African Egg', 2006, None), ('Ali Forever', 2006, None), ('Alone Trip', 2006, None), ('Amadeus Holy', 2006, None), ('American Circus', 2006, None), ('Amistad Midsummer', 2006, None), ('Armageddon Lost', 2006, None), ('Baked Cleopatra', 2006, None), ('Bang Kwai', 2006, None), ('Basic Easy', 2006, None), ('Berets Agent', 2006, None), ('B

## 4. How many rentals were in between 2005-07-01 and 2005-08-01?

In [17]:
import datetime

mapper_stmt4 = select(dict_table['rental']
                      ).where(and_(dict_table['rental'].columns.rental_date > datetime.datetime(2006, 7, 1),\
                                   dict_table['rental'].columns.rental_date < datetime.datetime(2006, 8, 1)))

print('Mapper select: ')
print(mapper_stmt4)

with engine.connect() as conn:
    result4 = conn.execute(mapper_stmt4).fetchall()

result4_length = len(result4)
print(result4)
print(result4_length)

Mapper select: 
SELECT rental.rental_id, rental.rental_date, rental.inventory_id, rental.customer_id, rental.return_date, rental.staff_id, rental.last_update 
FROM rental 
WHERE rental.rental_date > :rental_date_1 AND rental.rental_date < :rental_date_2
[]
0


## 5. How many rentals were in between 2010-01-01 and 2011-02-01?

In [18]:
import datetime

mapper_stmt5 = select(dict_table['rental']
                      ).where(and_(dict_table['rental'].columns.rental_date > datetime.datetime(2010, 1, 1),\
                                   dict_table['rental'].columns.rental_date < datetime.datetime(2010, 2, 1)))

print('Mapper select: ')
print(mapper_stmt5)

with engine.connect() as conn:
    result5 = conn.execute(mapper_stmt5).fetchall()

result5_length = len(result5)
print(result5)
print(result5_length)

Mapper select: 
SELECT rental.rental_id, rental.rental_date, rental.inventory_id, rental.customer_id, rental.return_date, rental.staff_id, rental.last_update 
FROM rental 
WHERE rental.rental_date > :rental_date_1 AND rental.rental_date < :rental_date_2
[]
0


## 6. Find the biggest payment in the rental.

In [19]:
# mapper_stmt6 = 'SELECT max(payment.amount) from payment'

mapper_stmt6 = select(dict_table['payment'].columns.amount).order_by(dict_table['payment'].columns.amount.desc()).limit(1)

print('Mapper select: ')
print(mapper_stmt6)

with engine.connect() as conn:
    result6 = conn.execute(mapper_stmt6).fetchall()

print(result6)

Mapper select: 
SELECT payment.amount 
FROM payment ORDER BY payment.amount DESC
 LIMIT :param_1
[(Decimal('11.99'),)]


## 7. Find all customers from Poland, Nigeria or Bangladesh.

In [20]:
mapper_stmt7 = text("""
    SELECT customer.first_name, customer.last_name, country.country 
    FROM customer
    INNER JOIN address ON address.address_id = customer.address_id
    INNER JOIN city ON city.city_id = address.city_id
    INNER JOIN country ON country.country_id = city.country_id
    WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh')
    GROUP BY customer.first_name, customer.last_name, country.country
""")

print('Mapper select:')
print(mapper_stmt7)

with engine.connect() as conn:
    result7 = conn.execute(mapper_stmt7).fetchall()

print(result7)

Mapper select:

    SELECT customer.first_name, customer.last_name, country.country 
    FROM customer
    INNER JOIN address ON address.address_id = customer.address_id
    INNER JOIN city ON city.city_id = address.city_id
    INNER JOIN country ON country.country_id = city.country_id
    WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh')
    GROUP BY customer.first_name, customer.last_name, country.country

[('Bertha', 'Ferguson', 'Nigeria'), ('Brian', 'Wyman', 'Poland'), ('Carol', 'Garcia', 'Nigeria'), ('Constance', 'Reid', 'Nigeria'), ('Elsie', 'Kelley', 'Nigeria'), ('Frank', 'Waggoner', 'Bangladesh'), ('Gladys', 'Hamilton', 'Nigeria'), ('Jimmie', 'Eggleston', 'Poland'), ('Jo', 'Fowler', 'Nigeria'), ('Johnnie', 'Chisholm', 'Poland'), ('Leah', 'Curtis', 'Poland'), ('Marilyn', 'Ross', 'Nigeria'), ('Marjorie', 'Tucker', 'Poland'), ('Michelle', 'Clark', 'Bangladesh'), ('Olga', 'Jimenez', 'Nigeria'), ('Rodney', 'Moeller', 'Nigeria'), ('Ruben', 'Geary', 'Poland'), ('Russell', '

Alternative:

In [21]:
mapper_stmt7 = select(dict_table['customer'].columns.first_name,
                       dict_table['customer'].columns.last_name,
                       dict_table['country'].columns.country).\
select_from(dict_table['customer']
        .join(
            dict_table['address'],
            dict_table['address'].columns.address_id == dict_table['customer'].columns.address_id)
        .join(
            dict_table['city'],
            dict_table['city'].columns.city_id == dict_table['address'].columns.city_id)
        .join(
            dict_table['country'],
            dict_table['country'].columns.country_id == dict_table['city'].columns.country_id)).\
where(or_(\
            and_(dict_table['country'].columns.country == 'Poland'),
            and_(dict_table['country'].columns.country == 'Nigeria'),
            and_(dict_table['country'].columns.country == 'Bangladesh'))).\
group_by(dict_table['customer'].columns.first_name,
            dict_table['customer'].columns.last_name,
            dict_table['country'].columns.country)


print('Mapper select: ')
print(mapper_stmt7)

with engine.connect() as conn:
    result7 = conn.execute(mapper_stmt7).fetchall()

print(result7)

Mapper select: 
SELECT customer.first_name, customer.last_name, country.country 
FROM customer JOIN address ON address.address_id = customer.address_id JOIN city ON city.city_id = address.city_id JOIN country ON country.country_id = city.country_id 
WHERE country.country = :country_1 OR country.country = :country_2 OR country.country = :country_3 GROUP BY customer.first_name, customer.last_name, country.country
[('Bertha', 'Ferguson', 'Nigeria'), ('Brian', 'Wyman', 'Poland'), ('Carol', 'Garcia', 'Nigeria'), ('Constance', 'Reid', 'Nigeria'), ('Elsie', 'Kelley', 'Nigeria'), ('Frank', 'Waggoner', 'Bangladesh'), ('Gladys', 'Hamilton', 'Nigeria'), ('Jimmie', 'Eggleston', 'Poland'), ('Jo', 'Fowler', 'Nigeria'), ('Johnnie', 'Chisholm', 'Poland'), ('Leah', 'Curtis', 'Poland'), ('Marilyn', 'Ross', 'Nigeria'), ('Marjorie', 'Tucker', 'Poland'), ('Michelle', 'Clark', 'Bangladesh'), ('Olga', 'Jimenez', 'Nigeria'), ('Rodney', 'Moeller', 'Nigeria'), ('Ruben', 'Geary', 'Poland'), ('Russell', 'Brinson'

## 8. Where staff memebers live?

In [22]:
mapper_stmt8 = text("""
    SELECT staff.first_name AS name, staff.last_name AS surname, address.address AS street, 
           city.city AS cityName, country.country AS countryName 
    FROM staff 
    INNER JOIN address ON address.address_id = staff.address_id 
    INNER JOIN city ON city.city_id = address.city_id 
    INNER JOIN country ON country.country_id = city.country_id 
    GROUP BY name, surname, street, cityName, countryName
""")

print('Mapper select: ')
print(mapper_stmt8)

with engine.connect() as conn:
    result8 = conn.execute(mapper_stmt8).fetchall()

print(result8)

Mapper select: 

    SELECT staff.first_name AS name, staff.last_name AS surname, address.address AS street, 
           city.city AS cityName, country.country AS countryName 
    FROM staff 
    INNER JOIN address ON address.address_id = staff.address_id 
    INNER JOIN city ON city.city_id = address.city_id 
    INNER JOIN country ON country.country_id = city.country_id 
    GROUP BY name, surname, street, cityName, countryName

[('Jon', 'Stephens', '1411 Lillydale Drive', 'Woodridge', 'Australia'), ('Mike', 'Hillyer', '23 Workhaven Lane', 'Lethbridge', 'Canada')]


Alternative:

In [23]:
mapper_stmt8 = select(
        dict_table['staff'].columns.first_name.label('name'),
        dict_table['staff'].columns.last_name.label('surname'),
        dict_table['address'].columns.address.label('street'),
        dict_table['city'].columns.city.label('cityName'),
        dict_table['country'].columns.country.label('countryName')
    ).select_from(
        dict_table['staff']
        .join(dict_table['address'], dict_table['staff'].columns.address_id == dict_table['address'].columns.address_id)
        .join(dict_table['city'], dict_table['address'].columns.city_id == dict_table['city'].columns.city_id)
        .join(dict_table['country'], dict_table['city'].columns.country_id == dict_table['country'].columns.country_id)
    ).group_by(
        dict_table['staff'].columns.first_name,
        dict_table['staff'].columns.last_name,
        dict_table['address'].columns.address,
        dict_table['city'].columns.city,
        dict_table['country'].columns.country
    )

print('Mapper select: ')
print(mapper_stmt8)

with engine.connect() as conn:
    result8 = conn.execute(mapper_stmt8).fetchall()

print(result8)

Mapper select: 
SELECT staff.first_name AS name, staff.last_name AS surname, address.address AS street, city.city AS "cityName", country.country AS "countryName" 
FROM staff JOIN address ON staff.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id GROUP BY staff.first_name, staff.last_name, address.address, city.city, country.country
[('Jon', 'Stephens', '1411 Lillydale Drive', 'Woodridge', 'Australia'), ('Mike', 'Hillyer', '23 Workhaven Lane', 'Lethbridge', 'Canada')]


## 9. How many staff members live in Argentina or Spain?

In [24]:
mapper_stmt9 = text("""
    SELECT COUNT(*) FROM (
        SELECT staff.first_name AS staffName, staff.last_name AS surname, address.address AS street, 
               city.city AS cityName, country.country AS countryName 
        FROM staff 
        INNER JOIN address ON address.address_id = staff.address_id 
        INNER JOIN city ON city.city_id = address.city_id 
        INNER JOIN country ON country.country_id = city.country_id 
        GROUP BY staffName, surname, street, cityName, countryName
    ) AS temp_table 
    WHERE countryName = 'Argentina' OR countryName = 'Spain'
""")

print('Mapper select:')
print(mapper_stmt9)

with engine.connect() as conn:
    result9 = conn.execute(mapper_stmt9).fetchall()

print(result9)

Mapper select:

    SELECT COUNT(*) FROM (
        SELECT staff.first_name AS staffName, staff.last_name AS surname, address.address AS street, 
               city.city AS cityName, country.country AS countryName 
        FROM staff 
        INNER JOIN address ON address.address_id = staff.address_id 
        INNER JOIN city ON city.city_id = address.city_id 
        INNER JOIN country ON country.country_id = city.country_id 
        GROUP BY staffName, surname, street, cityName, countryName
    ) AS temp_table 
    WHERE countryName = 'Argentina' OR countryName = 'Spain'

[(0,)]


Alternative:

In [25]:
mapper_stmt9 = (
    select(
        dict_table['staff'].columns.first_name.label('staffName'), 
        dict_table['staff'].columns.last_name.label('surname'),
        dict_table['address'].columns.address.label('street'),
        dict_table['city'].columns.city.label('cityName'),
        dict_table['country'].columns.country.label('countryName')
    )
    .join(dict_table['address'], dict_table['staff'].columns.address_id == dict_table['address'].columns.address_id)
    .join(dict_table['city'], dict_table['address'].columns.city_id == dict_table['city'].columns.city_id)
    .join(dict_table['country'], dict_table['city'].columns.country_id == dict_table['country'].columns.country_id)
    .group_by(
        dict_table['staff'].columns.first_name, 
        dict_table['staff'].columns.last_name,
        dict_table['address'].columns.address,
        dict_table['city'].columns.city,
        dict_table['country'].columns.country
    )
    .where(
        dict_table['country'].columns.country.in_(['Argentina', 'Spain'])
    )
)

print('Mapper select:')
print(mapper_stmt9)

with engine.connect() as conn:
    result9 = conn.execute(mapper_stmt9).fetchall()

count = len(result9)

print(f"Count: {count}")

Mapper select:
SELECT staff.first_name AS "staffName", staff.last_name AS surname, address.address AS street, city.city AS "cityName", country.country AS "countryName" 
FROM staff JOIN address ON staff.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id 
WHERE country.country IN (__[POSTCOMPILE_country_1]) GROUP BY staff.first_name, staff.last_name, address.address, city.city, country.country
Count: 0


## 10. Which categories of the films were rented by clients?

In [26]:
mapper_stmt10 = text("""
    SELECT DISTINCT category.name 
    FROM category
    INNER JOIN film_category ON category.category_id = film_category.category_id
    INNER JOIN film ON film_category.film_id = film.film_id
    INNER JOIN inventory ON film.film_id = inventory.film_id
    INNER JOIN rental ON rental.inventory_id = inventory.inventory_id
    ORDER BY category.name
""")

print('Mapper select:')
print(mapper_stmt10)

with engine.connect() as conn:
    result10 = conn.execute(mapper_stmt10).fetchall()

print(result10)

Mapper select:

    SELECT DISTINCT category.name 
    FROM category
    INNER JOIN film_category ON category.category_id = film_category.category_id
    INNER JOIN film ON film_category.film_id = film.film_id
    INNER JOIN inventory ON film.film_id = inventory.film_id
    INNER JOIN rental ON rental.inventory_id = inventory.inventory_id
    ORDER BY category.name

[('Action',), ('Animation',), ('Children',), ('Classics',), ('Comedy',), ('Documentary',), ('Drama',), ('Family',), ('Foreign',), ('Games',), ('Horror',), ('Music',), ('New',), ('Sci-Fi',), ('Sports',), ('Travel',)]


Alternative:

In [27]:
mapper_stmt10 = (
    select(
        dict_table['category'].columns.name.label('category_name')
    )
    .join(dict_table['film_category'], dict_table['category'].columns.category_id == dict_table['film_category'].columns.category_id)
    .join(dict_table['film'], dict_table['film_category'].columns.film_id == dict_table['film'].columns.film_id)
    .join(dict_table['inventory'], dict_table['film'].columns.film_id == dict_table['inventory'].columns.film_id)
    .join(dict_table['rental'], dict_table['inventory'].columns.inventory_id == dict_table['rental'].columns.inventory_id)
    .group_by(dict_table['category'].columns.name)  # Grupa po nazwie kategorii
    .order_by(dict_table['category'].columns.name)
)

print('Mapper select:')
print(mapper_stmt10)

with engine.connect() as conn:
    result10 = conn.execute(mapper_stmt10).fetchall()

print(result10)

Mapper select:
SELECT category.name AS category_name 
FROM category JOIN film_category ON category.category_id = film_category.category_id JOIN film ON film_category.film_id = film.film_id JOIN inventory ON film.film_id = inventory.film_id JOIN rental ON inventory.inventory_id = rental.inventory_id GROUP BY category.name ORDER BY category.name
[('Action',), ('Animation',), ('Children',), ('Classics',), ('Comedy',), ('Documentary',), ('Drama',), ('Family',), ('Foreign',), ('Games',), ('Horror',), ('Music',), ('New',), ('Sci-Fi',), ('Sports',), ('Travel',)]


## 11. Find all categories of films rented in America.

In [28]:
mapper_stmt11 = text("""
    SELECT DISTINCT category.name 
    FROM category
    INNER JOIN film_category ON category.category_id = film_category.category_id
    INNER JOIN film ON film_category.film_id = film.film_id
    INNER JOIN inventory ON film.film_id = inventory.film_id
    INNER JOIN rental ON rental.inventory_id = inventory.inventory_id
    INNER JOIN customer ON rental.customer_id = customer.customer_id
    INNER JOIN address ON customer.address_id = address.address_id
    INNER JOIN city ON address.city_id = city.city_id
    INNER JOIN country ON city.country_id = country.country_id
    WHERE country.country = 'America'
    ORDER BY category.name
""")

print('Mapper select:')
print(mapper_stmt11)

with engine.connect() as conn:
    result11 = conn.execute(mapper_stmt11).fetchall()

length_task11 = len(result11)

print(length_task11)
print(result11)

Mapper select:

    SELECT DISTINCT category.name 
    FROM category
    INNER JOIN film_category ON category.category_id = film_category.category_id
    INNER JOIN film ON film_category.film_id = film.film_id
    INNER JOIN inventory ON film.film_id = inventory.film_id
    INNER JOIN rental ON rental.inventory_id = inventory.inventory_id
    INNER JOIN customer ON rental.customer_id = customer.customer_id
    INNER JOIN address ON customer.address_id = address.address_id
    INNER JOIN city ON address.city_id = city.city_id
    INNER JOIN country ON city.country_id = country.country_id
    WHERE country.country = 'America'
    ORDER BY category.name

0
[]


Alternative:

In [29]:
mapper_stmt11 = select(dict_table['category'].columns.name.label('category_name')).\
select_from(dict_table['category']
            .join(
                dict_table['film_category'],
                dict_table['category'].columns.category_id == dict_table['film_category'].columns.category_id)
            .join(
                dict_table['film'],
                dict_table['film_category'].columns.film_id == dict_table['film'].columns.film_id)
            .join(
                dict_table['inventory'],
                dict_table['film'].columns.film_id == dict_table['inventory'].columns.film_id)
            .join(
                dict_table['rental'],
                dict_table['inventory'].columns.inventory_id == dict_table['rental'].columns.inventory_id)
            .join(
                dict_table['customer'],
                dict_table['rental'].columns.customer_id == dict_table['customer'].columns.customer_id)
            .join(
                dict_table['address'],
                dict_table['customer'].columns.address_id == dict_table['address'].columns.address_id)
            .join(
                dict_table['city'],
                dict_table['address'].columns.city_id == dict_table['city'].columns.city_id)
            .join(
                dict_table['country'],
                dict_table['city'].columns.country_id == dict_table['country'].columns.country_id)).\
where(dict_table['country'].columns.country == 'America').\
group_by(dict_table['category'].columns.name).\
order_by(dict_table['category'].columns.name)

print('Mapper select:')
print(mapper_stmt11)

with engine.connect() as conn:
    result11 = conn.execute(mapper_stmt11).fetchall()

length_task11 = len(result11)

print(length_task11)
print(result11)

Mapper select:
SELECT category.name AS category_name 
FROM category JOIN film_category ON category.category_id = film_category.category_id JOIN film ON film_category.film_id = film.film_id JOIN inventory ON film.film_id = inventory.film_id JOIN rental ON inventory.inventory_id = rental.inventory_id JOIN customer ON rental.customer_id = customer.customer_id JOIN address ON customer.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id 
WHERE country.country = :country_1 GROUP BY category.name ORDER BY category.name
0
[]


## 12. Find all title of films where one of these actors played: ,,Olympia Pfeiffer", ,,Julia Zellweger" or ,,Ellen Presley"

In [30]:
mapper_stmt12 = text("""
    SELECT film.title, actor.first_name, actor.last_name 
    FROM film
    INNER JOIN film_actor ON film_actor.film_id = film.film_id
    INNER JOIN actor ON actor.actor_id = film_actor.actor_id
    WHERE (actor.first_name = 'Olympia' AND actor.last_name = 'Pfeiffer') OR
          (actor.first_name = 'Julia' AND actor.last_name = 'Zellweger') OR
          (actor.first_name = 'Ellen' AND actor.last_name = 'Presley')
    ORDER BY film.title
""")

print('Mapper select:')
print(mapper_stmt12)

with engine.connect() as conn:
    result12 = conn.execute(mapper_stmt12).fetchall()

result12_length = len(result12)

print(result12_length)
print(result12)

Mapper select:

    SELECT film.title, actor.first_name, actor.last_name 
    FROM film
    INNER JOIN film_actor ON film_actor.film_id = film.film_id
    INNER JOIN actor ON actor.actor_id = film_actor.actor_id
    WHERE (actor.first_name = 'Olympia' AND actor.last_name = 'Pfeiffer') OR
          (actor.first_name = 'Julia' AND actor.last_name = 'Zellweger') OR
          (actor.first_name = 'Ellen' AND actor.last_name = 'Presley')
    ORDER BY film.title

69
[('Badman Dawn', 'Olympia', 'Pfeiffer'), ('Bilko Anonymous', 'Ellen', 'Presley'), ('Breakfast Goldfinger', 'Julia', 'Zellweger'), ('Caribbean Liberty', 'Ellen', 'Presley'), ('Casper Dragonfly', 'Ellen', 'Presley'), ('Chitty Lock', 'Olympia', 'Pfeiffer'), ('Color Philadelphia', 'Olympia', 'Pfeiffer'), ('Contact Anonymous', 'Olympia', 'Pfeiffer'), ('Cranes Reservoir', 'Julia', 'Zellweger'), ('Dares Pluto', 'Julia', 'Zellweger'), ('Deep Crusade', 'Olympia', 'Pfeiffer'), ('Detective Vision', 'Julia', 'Zellweger'), ('Divorce Shining', 

Alternative:

In [31]:
mapper_stmt12 = select(dict_table['film'].columns.title,
                        dict_table['actor'].columns.first_name,
                        dict_table['actor'].columns.last_name).\
select_from(dict_table['film']
            .join(
                  dict_table['film_actor'],
                  dict_table['film_actor'].columns.film_id == dict_table['film'].columns.film_id)
            .join(
                  dict_table['actor'],
                  dict_table['actor'].columns.actor_id == dict_table['film_actor'].columns.actor_id)).\
where(or_(\
            and_(dict_table['actor'].columns.first_name == 'Olympia',
                 dict_table['actor'].columns.last_name == 'Pfeiffer'),
            and_(dict_table['actor'].columns.first_name == 'Julia',
                 dict_table['actor'].columns.last_name == 'Zellweger'),
            and_(dict_table['actor'].columns.first_name == 'Ellen',
                 dict_table['actor'].columns.last_name == 'Presley'))).\
order_by(dict_table['film'].columns.title)

print('Mapper select:')
print(mapper_stmt12)

with engine.connect() as conn:
    result12 = conn.execute(mapper_stmt12).fetchall()

result12_length = len(result12)

print(result12_length)
print(result12)

Mapper select:
SELECT film.title, actor.first_name, actor.last_name 
FROM film JOIN film_actor ON film_actor.film_id = film.film_id JOIN actor ON actor.actor_id = film_actor.actor_id 
WHERE actor.first_name = :first_name_1 AND actor.last_name = :last_name_1 OR actor.first_name = :first_name_2 AND actor.last_name = :last_name_2 OR actor.first_name = :first_name_3 AND actor.last_name = :last_name_3 ORDER BY film.title
69
[('Badman Dawn', 'Olympia', 'Pfeiffer'), ('Bilko Anonymous', 'Ellen', 'Presley'), ('Breakfast Goldfinger', 'Julia', 'Zellweger'), ('Caribbean Liberty', 'Ellen', 'Presley'), ('Casper Dragonfly', 'Ellen', 'Presley'), ('Chitty Lock', 'Olympia', 'Pfeiffer'), ('Color Philadelphia', 'Olympia', 'Pfeiffer'), ('Contact Anonymous', 'Olympia', 'Pfeiffer'), ('Cranes Reservoir', 'Julia', 'Zellweger'), ('Dares Pluto', 'Julia', 'Zellweger'), ('Deep Crusade', 'Olympia', 'Pfeiffer'), ('Detective Vision', 'Julia', 'Zellweger'), ('Divorce Shining', 'Julia', 'Zellweger'), ('Effect Gladiator